### Implementing emotion cause analysis using Bert model and adding additional layers

In [1]:
# Imports for most of the notebook
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List
import random
from tqdm.autonotebook import tqdm

C:\Users\vivek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
print(torch.cuda.is_available())
device = torch.device("cpu")

True


### Loading the data

In [3]:
import json
# Opening JSON file
f = open('.\Data\\text\Subtask_1_train.json')
# returns JSON object as 
# a dictionary
data = json.load(f)
#data

### separate labels and data

In [4]:
## todo: find x_data and y_label
x_data = []
y_cause_labels = []
y_emotion_labels = []

# Iterate through each conversation in the dataset
for conv in data:
    # Extract conversation, emotion-cause pairs, and emotion labels
    conversation = conv['conversation']
    emotion_cause_pairs = conv['emotion-cause_pairs']
    
    # Extract emotion labels from each utterance in the conversation
    emotion_labels = [utterance['emotion'] for utterance in conversation]

    # Append to the respective lists
    x_data.append(conversation)
    y_cause_labels.extend(emotion_cause_pairs)
    y_emotion_labels.append(emotion_labels) # figure out if its append or extend

# Print the extracted data
# print("Conversations:")
# for conv in x_data:
#     print(conv)

# print("\nEmotion-Cause Pairs:")
# for pair in y_cause_labels:
#     print(pair)

# print("\nEmotion Labels:")
# print(y_emotion_labels)

In [5]:
#x_data

### Change the label formart

In [6]:
## todo: change ['17_anger', '16_we could throw you both in'] --> ['17_anger', '16'] and making sure labels is list[list[labels for each conversation]]. confirm if labels needs to be in this formart list[list[labels for each conversation]]

## Create Train and Dev Set

In [7]:
# from sklearn.model_selection import train_test_split
# train_conv, dev_conv = train_test_split(data, test_size=0.2, random_state=42)

#todo: with above x_data and y_label get x_train, y_ label, x_dev and y_dev

from sklearn.model_selection import train_test_split

# Split the data into training and development sets
x_train, x_dev, y_train, y_dev = train_test_split(x_data, y_emotion_labels, test_size=0.2, random_state=42)

# Print the sizes of the sets
print("Training set size:", len(x_train))
print("Development set size:", len(x_dev))


Training set size: 1099
Development set size: 275


In [8]:
print(len(y_train[0]))

11


### bert tokenizer

In [9]:
# Use this for now

# class BatchTokenizer:
#     """Tokenizes and pads a batch of input sentences."""

#     def __init__(self, model_name="bert-base-uncased"):
#         """Initializes the tokenizer

#         Args:
#             pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
#         """
#         self.conv_tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model_name = model_name
    
#     def get_sep_token(self,):
#         return self.conv_tokenizer.sep_token
    
#     def __call__(self, conv_batch: List[dict]) -> List[dict[str]]:
#         """Uses the huggingface tokenizer to tokenize and pad a batch.

#         We return a dictionary of tensors per the huggingface model specification.

#         Args:
#             batch (List[str]): A List of sentence strings

#         Returns:
#             Dict: The dictionary of token specifications provided by HuggingFace
#         """
#         # The HF tokenizer will PAD for us, and additionally combine 
#         # The two sentences deimited by the [SEP] token.
#         # combined_texts = [f"{utterance['speaker']}: {utterance['text']}" for utterance in conv_batch["conversation"]]
#         # print(combined_texts)
        
#         # print(conv_batch)
#         combined_single_texts=[]
#         batch_texts=[]
#         for conv in conv_batch:
#             #print(conv)
#             combined_single_texts=[]
#             for utterance in conv:
#                 combined_single_texts.append(f"{utterance['speaker']}: {utterance['text']}")
#             batch_texts.append(combined_single_texts)
#         #print(batch_texts)
#         encoded=[]
#         for batch in batch_texts:
#             enc = self.conv_tokenizer(
#                 batch,
#                 padding=True,
#                 return_token_type_ids=False,
#                 return_tensors='pt'
#             )
#             encoded.append(enc)
#         #print(encoded)
#         return enc
    

# HERE IS AN EXAMPLE OF HOW TO USE THE BATCH TOKENIZER
# tokenizer = BatchTokenizer()
# x = tokenizer(x_train[0:10])
#print(x)
#tokenizer.conv_tokenizer.batch_decode(x["input_ids"])

In [13]:
# from typing import List, Dict
from transformers import AutoTokenizer

class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self, model_name="bert-base-uncased"):
        """Initializes the tokenizer.

        Args:
            model_name (str, optional): Pretrained model name. Defaults to "bert-base-uncased".
        """
        self.conv_tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model_name = model_name
    
    def get_sep_token(self):
        return self.conv_tokenizer.sep_token
    
    def __call__(self, conv_batch: List[Dict[str, List[Dict[str, str]]]]) -> Dict[str, torch.Tensor]:
        """Uses the Hugging Face tokenizer to tokenize and pad a batch.

        Args:
            conv_batch (List[Dict[str, List[Dict[str, str]]]]): A list of conversations.

        Returns:
            Dict[str, torch.Tensor]: The dictionary of token specifications provided by Hugging Face.
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # the sentences delimited by the [SEP] token.
        
        tokenized_conversations = []

        for conv in conv_batch:
            conv_tokens = self.conv_tokenizer(
                [f"{utterance['speaker']}: {utterance['text']}" for utterance in conv],
                padding=True,
                truncation=True,
                return_tensors="pt",
                return_token_type_ids=False,
                max_length=512  
            )
            tokenized_conversations.append(conv_tokens)

        return tokenized_conversations

# Example of how to use the batch tokenizer
tokenizer = BatchTokenizer()
batch_encodings = tokenizer([x_train[0], x_train[1]])  # Pass a batch of conversations
#print(batch_encodings)
decoded_texts = tokenizer.conv_tokenizer.batch_decode(batch_encodings[1]["input_ids"])
print("Decoded Texts for Conversation 1:", decoded_texts)


Decoded Texts for Conversation 1: ['[CLS] monica : ow! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] richard : really?! well, it is just like everyone else apartment. it is got rooms, walls, and ceilings. [SEP]', "[CLS] richard's date : well, i just wanted to see where you lived. now, give me the tour. [SEP] [PAD] [PAD]", '[CLS] monica : oh my god! oh my god! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] richard : ah well, this is the living room. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] richard : all right. this is the kitchen. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] richard : the bedroom. well it is pretty much your typical... bedroom. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', "[CLS] richard's date

In [14]:
#len(x_train)

### Making batched of the input

In [15]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        #print(lst[i:i + n])
        yield lst[i:i + n]

batch_size = 8
        
# Notice that since we use huggingface, we tokenize and
# encode in all at once!
tokenizer = BatchTokenizer()
train_input_batches_temp = [b for b in chunk(x_train, batch_size)]
#print(train_input_batches)
# Tokenize + encode
train_input_batches = [tokenizer(batch) for batch in train_input_batches_temp]

dev_input_batches = [b for b in chunk(x_dev, batch_size)]

# Tokenize + encode
dev_input_batches = [tokenizer(batch) for batch in dev_input_batches]


In [16]:
#print(dev_input_batches[1])

In [17]:
#todo: batch and encode the following needed for model training  x_train, y_ label, x_dev and y_dev

def encode_labels(batch_labels: List[List[str]], label_map: Dict[str, int]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor using the provided label map

    Args:
        batch_labels (List[List[str]]): List of lists of labels in the batch
        label_map (Dict[str, int]): Dictionary mapping label strings to integers

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    # Encode each label in the batch using the label map
    encoded_labels = [torch.tensor([label_map[label] for label in labels], dtype=torch.float32) for labels in batch_labels]
    #print(encoded_labels)

    # Convert the list of lists to a tensor
  

    return encoded_labels
# unique emotions ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
encode_label_map = {'neutral': 0, 'surprise': 1, 'anger': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'fear': 6}

train_label_batches = [b for b in chunk(y_emotion_labels, batch_size)]
# print(train_label_batches[0])
# for batch in train_label_batches:
#     print(batch)
train_label_batches = [encode_labels(batch, encode_label_map) for batch in train_label_batches]

dev_batch_labels = [b for b in chunk(y_dev, batch_size)]
dev_batch_labels = [encode_labels(batch, encode_label_map) for batch in dev_batch_labels]

### Evaluation metric

In [18]:
import numpy as np
from numpy import sum as t_sum
from numpy import logical_and


def precision(predicted_labels, true_labels, which_label=1):
    """
    Precision is True Positives / All Positives Predictions
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(pred_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def recall(predicted_labels, true_labels, which_label=1):
    """
    Recall is True Positives / All Positive Labels
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(true_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def f1_score(
    predicted_labels: List[int],
    true_labels: List[int],
    which_label: int
):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)
    
    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.


def macro_f1(
    predicted_labels: List[int],
    true_labels: List[int],
    possible_labels: List[int],
    label_map=None
):
    converted_prediction = [label_map[int(x)] for x in predicted_labels] if label_map else predicted_labels
    scores = [f1_score(converted_prediction, true_labels, l) for l in possible_labels]
    # Macro, so we take the uniform avg.
    return sum(scores) / len(scores)

### Model for emotion classifier

In [19]:
class EmotionClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int, model_name='prajjwal1/bert-small'):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained(model_name)
        
        # [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive. 
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        
        self.bert_hidden_dimension = self.bert.config.hidden_size
        
        # DONE: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size. Hint: torch.nn.Linear()
        
        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, self.hidden_size)
        
        # DONE: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
        
        self.relu = torch.nn.ReLU()
        
        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
    
    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols BERT.
            Then, get CLS represenation.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: CLS token embedding
        """
        # Get the contextualized embedding for each input symbol
        encoded_sequence = self.bert(**symbols)
        
        # Extract the [CLS] token representation
        cls_token_embedding = encoded_sequence.last_hidden_state[:, 0, :]  # Extract [CLS] token
        
        return cls_token_embedding.unsqueeze(1)  # Reshape to batch_size x 1 x bert_hidden_dimension

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        """Forward pass."""
        # Directly get BERT output (including [CLS] token) from input symbols
        encoded_sents = self.encode_text(symbols)
        output = self.hidden_layer(encoded_sents)
        # Extract the [CLS] token representation
        #cls_token_embedding = encoded_sequence.last_hidden_state[:, 0, :]  # Assuming [CLS] is at index 0
        
        # Pass through hidden layer and activation
        #output = self.hidden_layer(cls_token_embedding.unsqueeze(1))
        output = self.hidden_layer(output)
        output = self.relu(output)
        
        # Classification layer
        output = self.classifier(output)
        return self.log_softmax(output)

In [20]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents)
    return list(torch.argmax(logits, axis=2).squeeze().numpy())

### Training loop

In [21]:
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
):
    print("Training...")
    #loss_func = torch.nn.NLLLoss()
    loss_func = torch.nn.MSELoss()
    
    
    batches = list(zip(train_features, train_labels))
    #print(batches)
    random.shuffle(batches)
    # Convert your data into PyTorch tensors
    #train_features_tensor = torch.tensor(train_features)
    #train_labels_tensor = torch.tensor(train_labels)
    
    # Create a TensorDataset from your features and labels
    #train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
    
    # Create a DataLoader to handle batching and shuffling
    #train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    for i in range(num_epochs):
        losses = []
        
        for features, labels in tqdm(batches):
            # print(features, labels)
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features).squeeze(1)

            # loss = loss_func(preds, labels.to(device))
            loss = loss_func(preds, labels)
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            # pred = predict(model, sents).cpu()
            pred = predict(model, sents)
            all_preds.extend(pred)
            # all_labels.extend(list(labels.cpu().numpy()))
            all_labels.extend(list(labels.numpy()))

        x = np.array(all_labels)
        unique_possible_labels = list(np.unique(x))
        dev_f1 = macro_f1(all_preds, all_labels, unique_possible_labels)
        # dev_f1 = macro_f1_score(all_preds, all_labels, list(train_labels))
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model

In [22]:
# You can increase epochs if need be
epochs = 20

# TODO: Find a good learning rate and hidden size
LR = 0.01
hidden_size = 10

#possible_labels = set(train_labels)
model = EmotionClassifier(output_size=7, hidden_size=hidden_size)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), LR)


training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    dev_input_batches,
    dev_batch_labels,
    optimizer,
    model,
)

Training...


  0%|          | 0/138 [00:00<?, ?it/s]

TypeError: BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 512, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=512, out_features=512, bias=True)
            (key): Linear(in_features=512, out_features=512, bias=True)
            (value): Linear(in_features=512, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=512, out_features=512, bias=True)
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
batches